In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487-final-project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

['eval.py', 'README.md', 'test.py', 'naivebayes.py', 'scrape.py', 'preprocess.py', 'requirements.txt', 'data', '__pycache__', 'model.py', 'checkpoint.pth', 'checkpoint_7.pth', 'train.ipynb']


In [3]:
!pip install transformers
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
from google.colab import files

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 6.5 MB 49.4 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from model import train
from model import BertClassifier

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Dataset

In [5]:
np.random.seed(112)

df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'data/MBIC/labeled_dataset.csv'))

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

# Hyperparameter Tuning

In [ ]:
epoch_list = [3, 5, 10, 15, 20]
learning_rate_list = [1e-6, 1e-5, 1e4, 1e-3, 1e-2, 1e-1]
bert_clf = BertClassifier()
for epoch_value in epoch_list:
    for learning_rate in learning_rate_list:
        train(bert_clf, df_train, df_val, learning_rate, epoch_value)
        

# Train Model

In [6]:
EPOCHS = 10
clf = BertClassifier()
LR = 1e-6

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train(clf, df_train, df_val, LR, EPOCHS)

torch.save(clf.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_7.pth'))

files.download(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_7.pth'))

Using Cuda: True


100%|██████████| 680/680 [01:32<00:00,  7.34it/s]


Epochs: 1 | Train Loss:  0.528         | Train Accuracy:  0.483         | Val Loss:  0.506         | Val Accuracy:  0.518


100%|██████████| 680/680 [01:32<00:00,  7.38it/s]


Epochs: 2 | Train Loss:  0.470         | Train Accuracy:  0.645         | Val Loss:  0.469         | Val Accuracy:  0.582


...

Epochs: 9 | Train Loss:  0.110         | Train Accuracy:  0.971         | Val Loss:  0.207         | Val Accuracy:  0.871


100%|██████████| 680/680 [01:32<00:00,  7.38it/s]


Epochs: 10 | Train Loss:  0.085         | Train Accuracy:  0.979         | Val Loss:  0.205         | Val Accuracy:  0.871


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test Model

In [12]:
from model import Dataset

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model_test = BertClassifier()
state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint.pth'))
model_test.load_state_dict(state_dict)
model_test.to(device)

test_data = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=2, shuffle=True)

total_acc_test = 0
with torch.no_grad():
  for test_input, test_label in tqdm(test_dataloader):

      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model_test(input_id, mask)
      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc
      
print()
print("Test Accuracy of", total_acc_test/len(df_test))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 85/85 [00:03<00:00, 23.81it/s]


Test Accuracy of 0.9176470588235294
